# Arbitrary Style Transfer
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/ghiasi-arbitrary-style-transfer/blob/main/arbitrary-style-transfer.ipynb)

This notebook lets you try out image stylization using the approach described
by Ghiasi, Lee, Kudlur, Dumoulin, and Shlens in their article _Exploring the
Structure of a Real-Time, Arbitrary Neural Artistic Stylization Network_.  For
more details, see my github repositories [`ghiasi-arbitrary-style-transfer`](
https://github.com/mdehling/ghiasi-arbitrary-style-transfer) and
[`nstesia`](https://github.com/mdehling/nstesia).

In [ ]:
# At the time of writing, the versions of pip and setuptools provided by colab
# do not have full `pyproject.toml` support --- they must be updated before
# installing the nstesia package.  This cell will do just that.
#
# Once this cell has finished running, YOU MUST RESTART THE COLAB RUNTIME.
try:
    from google import colab

    # Pull everything else from the repository in to the Colab environment.
    !git init .
    !git remote add origin https://github.com/mdehling/ghiasi-arbitrary-style-transfer.git
    !git pull --depth=1 origin main

    # These are just to avoid some scary-looking (but harmless) error messages.
    !pip uninstall -q -y numba
    !pip install -q 'jedi>=0.10'

    # Minimum versions required for PEP-660 support.
    !pip install -q 'pip>=21.3' 'setuptools>=64'

    # Finally install the one we came for.
    !pip install -q -r requirements.txt

except ImportError:
    pass

In [ ]:
from IPython.display import display

from os import environ as env
env['TF_CPP_MIN_LOG_LEVEL'] = '2'               # hide info & warnings
env['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'       # grow GPU memory as needed

import tensorflow as tf
import nstesia as nst

## Loading the Model

In [ ]:
# This will download and unpack the saved model.
![ ! -e saved ] && \
    gdown 1rZrUQLriaJhsYFFUojotm5LRiCNikow5 && \
    tar xjf saved.tar.bz2 && \
    rm -f saved.tar.bz2

In [ ]:
model = nst.ghiasi_2017.StyleTransferModel.from_saved('saved/model-16')

## Simple Image Stylization

In [ ]:
def show_image(image_tensor):
    display(tf.keras.utils.array_to_img(tf.squeeze(image_tensor,axis=0)))

In [ ]:
content_image = nst.io.load_image('img/content/chicago.jpg')
style_image = nst.io.load_image('img/style/wescoat-candy.jpg')
pastiche_image = model( (content_image, style_image) )
show_image(pastiche_image)

In [ ]:
content_image = nst.io.load_image('img/content/sunflower.jpg')
pastiche_images = [
    model((content_image, style_vector([i]))) for i in range(n_styles)
]
grid_image = nst.image.grid(pastiche_images, ncols=8)
show_image(grid_image)

In [ ]:
grid_image = nst.image.grid(model.style_images, ncols=8)
show_image(grid_image)

## Content-Style Grids

In [ ]:
def content_style_grid(image_files, styles):
    """
    Show a grid of stylizations with content images as rows, styles as columns.

    Args:
        images_files:
            A list of strings.  The file names of the content images to load.
        styles:
            A list of ints.  The indices of the styles to use.

    Returns:
        An image representing the grid of stylizations.
    """
    content_images = [ nst.io.load_image(file) for file in image_files ]

    images = [None] + [model.style_images[index] for index in styles]
    for content_image in content_images:
        images += [content_image] + [
            model((content_image,style_vector([index]))) for index in styles
        ]

    return nst.image.grid(images, ncols=len(styles)+1)

In [ ]:
# img/results/content-style-matrix-1.png
grid_image = content_style_grid(
    ['img/content/sunflower.jpg', 'img/content/bochum.jpg'],
    styles=[31, 16, 15],
)
show_image(grid_image)

In [ ]:
# img/results/content-style-matrix-2.png
grid_image = content_style_grid(
    ['img/content/brad.jpg', 'img/content/karya.jpg'],
    styles=[14, 1, 13],
)
show_image(grid_image)

## Mixing Styles

In [ ]:
def style_mix_matrix(content_image, styles, nrows=5, ncols=5):
    """
    Create a Matrix of Mixed Stylizations.

    Args:
        content_image:
            A 4-D tensor of shape `[1,H,W,3]`.
        styles:
            A 4-tuple of style indices in `0..n_styles`.
        nrows:
            An integer.  The number of rows of the generated matrix.
        ncols:
            An integer.  The number of columns of the generated matrix.

    Returns:
        A 4-D tensor of shape `[1,H',W',3]` representing the mixed style matrix.
    """
    pastiche_images = []
    for i in range(nrows):
        for j in range(ncols):
            # weighted average of the 4 style basis vectors
            v = style_vector(styles,
                [(nrows-i-1)*(ncols-j-1), (nrows-i-1)*j, i*(ncols-j-1), i*j]
            ) / ( (nrows-1)*(ncols-1) )
            pastiche_images.append(
                model( (content_image,v) )
            )

    return nst.image.grid(pastiche_images, ncols=ncols)

In [ ]:
# img/results/style-mix-matrix.png
content_image = nst.io.load_image('img/content/brad.jpg')
style_mix_image = style_mix_matrix(content_image, [9,17,23,28], nrows=4, ncols=4)
show_image(style_mix_image)